<a href="https://colab.research.google.com/github/paris1c/big-data-challenge/blob/main/Level2/level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
#spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.

In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-03 22:26:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.25MB/s    in 0.7s    

2022-09-03 22:26:08 (1.25 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [9]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
games_df.show(5)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [10]:
filtered_games_df = games_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
filtered_games_df.show(5)


+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          1|   N|                Y|
|          3|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [12]:
# Filter for greater than 50 total votes
total_votes_df = filtered_games_df.filter(filtered_games_df["total_votes"] >= 50)
total_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|          147|        175|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
|          1|            7|        139|   N|                N|
|          1|           34|        116|   N|                N|
|          5|           88|        110|   N|                N|
|          2|           99|        119|   N|                N|
|          1|          102|        106|   N|                N|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [13]:
# Filter for greater than 50% helpful vote percentage
helpful_votes_df = total_votes_df.filter(total_votes_df["helpful_votes"]/total_votes_df["total_votes"] >= 0.5)
helpful_votes_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|          147|        175|   N|                Y|
|          2|           55|         60|   N|                N|
|          1|           51|         65|   N|                Y|
|          2|          151|        198|   N|                N|
|          1|           49|         51|   N|                Y|
|          5|           88|        110|   N|                N|
|          2|           99|        119|   N|                N|
|          1|          102|        106|   N|                N|
|          1|           79|         91|   N|                N|
|          2|           44|         53|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [14]:
paid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'Y')
paid_df.describe().show(10)

+-------+-----------------+------------------+------------------+----+-----------------+
|summary|      star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+-----------------+------------------+------------------+----+-----------------+
|  count|               34|                34|                34|  34|               34|
|   mean|4.294117647058823|104.23529411764706|115.97058823529412|null|             null|
| stddev|0.798840514469523|   88.943311474059|   92.314182007119|null|             null|
|    min|                2|               111|               102|   Y|                N|
|    max|                5|                97|                88|   Y|                N|
+-------+-----------------+------------------+------------------+----+-----------------+



In [15]:
# Unpaid reviews
unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine']== 'N')
unpaid_df.describe().show()

+-------+-----------------+-----------------+------------------+-----+-----------------+
|summary|      star_rating|    helpful_votes|       total_votes| vine|verified_purchase|
+-------+-----------------+-----------------+------------------+-----+-----------------+
|  count|            10988|            10988|             10988|10988|            10988|
|   mean|3.479887149617765|110.9690571532581|127.31461594466691| null|             null|
| stddev|1.610484756356454|212.5455746971236| 229.3795520922815| null|             null|
|    min|                1|              100|               100|    N|                N|
|    max|                5|              999|               999|    N|                Y|
+-------+-----------------+-----------------+------------------+-----+-----------------+



In [16]:
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = round(float(paid_five_star_number) / float(paid_number),4)
print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')


Number of paid reviews 34
Number of paid five star reviews 16
Percantage of paid reviews that are five stars 47.06%


In [17]:
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = round(float(unpaid_five_star_number) / float(unpaid_number),4)
print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 10988
Number of unpaid five star reviews 4585
Percantage of paid reviews that are five stars 41.730000000000004%


**About Amazon Vine?** https://www.amazon.com/vine/about?ie=UTF8
Amazon Vine is an invitation-only program which selects the most insightful reviewers in the Amazon store to serve as Vine Voices. Vine Voices have the unique opportunity to order items free of charge and share their product experiences with Amazon customers to help them make informed buying decisions.

**Analysis:**
As we see the posibility of getting a 5 Star review from paid_reviews are higher compared to up-paid reviews.